In [46]:
import pandas as pd
import geopandas as gpd
import os
import requests
import yaml

Aggrégation des données

In [ ]:
# folder = "../raw_data/year_data/"
# dff = []
# files = [file for file in os.listdir(folder) if file.endswith('.csv')]
# for file in files:
#     file_path = os.path.join(folder, file)
#     if file in ['caracteristiques_2018.csv', 'caracteristiques_2008.csv',
#                 'caracteristiques_2011.csv','caracteristiques_2010.csv',
#                 'caracteristiques_2005.csv','caracteristiques_2006.csv',
#                 'caracteristiques_2007.csv',
#                 'caracteristiques_2013.csv','caracteristiques_2012.csv',
#                 'caracteristiques_2014.csv','caracteristiques_2015.csv',
#                 'caracteristiques_2016.csv','caracteristiques_2017.csv']:
#         encoding = 'latin-1'
#         sep = ','
#     elif file in ['caracteristiques_2009.csv']:
#         encoding = 'latin-1'
#         sep = '\t'
#     elif file in ['vehicules_2019.csv','vehicules_2020.csv','vehicules_2021.csv','vehicules_2022.csv','lieux_2019.csv','lieux_2020.csv','lieux_2021.csv',
#                   'lieux_2022.csv','caracteristiques_2019.csv',
#                   'caracteristiques_2021.csv','caracteristiques_2022.csv',
#                   'caracteristiques_2020.csv' ]:
#         encoding = 'latin-1'

#         sep = ';'
#     elif file in ['2019.csv', '2020.csv', '2021.csv', '2022.csv']:
#         sep = ';'
#         encoding = 'latin-1'
#     else:
#         sep = ','
#         encoding = 'utf-8'
#     print(file_path)
#     df = pd.read_csv(file_path, sep=sep, encoding=encoding)
#     # df = df.applymap(lambda x: str(x).replace('"', ''))
#     dff.append(df)
#     df_chiant = pd.concat(dff)


API POUR REQUEST & SAVE URL


In [ ]:
### call api for datasets urls


def get_datasets_url(url=None):
    url ='https://www.data.gouv.fr/api/1/datasets/53698f4ca3a729239d2036df/'
    r = requests.get(url).json()
    return {el['title']: el['latest'] for el in r['resources'] if el['title'].endswith(".csv") and not el['title'].startswith("vehicules-immatricules") }

lieux_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("lieux")}
usagers_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("usagers")}
car_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("car")}
vehicule_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("vehicule")}

all_urls = [lieux_datasets, usagers_datasets, car_datasets,vehicule_datasets]

### download csvs if not already

for url_dict in all_urls:
    for path, url in url_dict.items():
        path = '../raw_data/' + path
        if not os.path.exists(path):
            response = requests.get(url)
            if response.status_code == 200:
                with open(path, 'wb') as f:
                    f.write(response.content)

### check datasets

folder = "../raw_data/"
os.listdir(folder)

In [ ]:
chemin_fichier_yml = '../config.yml'
chemin_dossier = '../raw_data/'


with open (chemin_fichier_yml, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    rename_config = config.get('rename')

for old_name, new_name in rename_config.items():
    chemin_ancien_fichier = os.path.join(chemin_dossier, old_name)
    chemin_nouveau_fichier = os.path.join(chemin_dossier, new_name)

    if os.path.exists(chemin_ancien_fichier):
        os.rename(chemin_ancien_fichier, chemin_nouveau_fichier)
        print(f"Fichier renommé : {old_name} -> {new_name}")
    else:
        print(f"Fichier non trouvé : {old_name}")


In [74]:


def concat_files(starting_word):

    chemin_fichier_yml = '../config.yml'
    with open(chemin_fichier_yml, 'r') as f:
        config = yaml.safe_load(f)
        config_sep = config.get('sep')
        config_encoding = config.get('encoding')

    chemin_dossier = '../raw_data/'

    df_concat = pd.DataFrame()
    files = [file for file in os.listdir(chemin_dossier) if file.endswith('.csv') and file.startswith(starting_word)]

    print(files)
    for file in files:
        chemin_fichier = os.path.join(chemin_dossier, file)

        if file in config_sep:
            sep = config_sep[file]
        else:
            sep = ','

        if file in config_encoding:
            encoding = config_encoding[file]
        else:
            encoding = 'utf-8'

        df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)

        df_concat = pd.concat([df_concat, df1])

    return df_concat


In [79]:
carac_df = concat_files("caracteristiques")
lieux_df = concat_files("lieux")
usager_df = concat_files("usagers")
vehi_df = concat_files("vehicules")

['caracteristiques_2018.csv', 'caracteristiques_2019.csv', 'caracteristiques_2022.csv', 'caracteristiques_2009.csv', 'caracteristiques_2021.csv', 'caracteristiques_2020.csv', 'caracteristiques_2008.csv', 'caracteristiques_2011.csv', 'caracteristiques_2005.csv', 'caracteristiques_2010.csv', 'caracteristiques_2006.csv', 'caracteristiques_2012.csv', 'caracteristiques_2013.csv', 'caracteristiques_2007.csv', 'caracteristiques_2017.csv', 'caracteristiques_2016.csv', 'caracteristiques_2014.csv', 'caracteristiques_2015.csv']


/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_9220/241619552.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


['lieux_2017.csv', 'lieux_2016.csv', 'lieux_2014.csv', 'lieux_2015.csv', 'lieux_2011.csv', 'lieux_2005.csv', 'lieux_2010.csv', 'lieux_2006.csv', 'lieux_2012.csv', 'lieux_2013.csv', 'lieux_2007.csv', 'lieux_2022.csv', 'lieux_2009.csv', 'lieux_2021.csv', 'lieux_2020.csv', 'lieux_2008.csv', 'lieux_2018.csv', 'lieux_2019.csv']


/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_9220/241619552.py:28: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)
/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_9220/241619552.py:28: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)
/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_9220/241619552.py:28: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)
/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_9220/241619552.py:28: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


['usagers_2006.csv', 'usagers_2012.csv', 'usagers_2013.csv', 'usagers_2007.csv', 'usagers_2011.csv', 'usagers_2005.csv', 'usagers_2010.csv', 'usagers_2014.csv', 'usagers_2015.csv', 'usagers_2017.csv', 'usagers_2016.csv', 'usagers_2018.csv', 'usagers_2019.csv', 'usagers_2021.csv', 'usagers_2009.csv', 'usagers_2008.csv', 'usagers_2020.csv', 'usagers_2022.csv']
['vehicules_2018.csv', 'vehicules_2019.csv', 'vehicules_2022.csv', 'vehicules_2009.csv', 'vehicules_2021.csv', 'vehicules_2020.csv', 'vehicules_2008.csv', 'vehicules_2005.csv', 'vehicules_2011.csv', 'vehicules_2010.csv', 'vehicules_2012.csv', 'vehicules_2006.csv', 'vehicules_2007.csv', 'vehicules_2013.csv', 'vehicules_2017.csv', 'vehicules_2016.csv', 'vehicules_2014.csv', 'vehicules_2015.csv']
